# Projeto de Previsão de Churn - Conclusão e Comparação Final dos Modelos

Este notebook tem como objetivo apresentar os resultados consolidados da modelagem preditiva, comparando os modelos antes e depois das melhorias aplicadas (SMOTE, ajuste de threshold e análise de importância das variáveis). Ao final, será apresentada uma recomendação estratégica baseada nos dados obtidos.


## 1️⃣ Importações e Preparação do Ambiente

In [1]:
# Bibliotecas principais
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Visualização de métricas
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score, roc_curve
)

# Estilo de visualização
sns.set(style='whitegrid')


## 2️⃣ Carregamento dos Arquivos de Resultados

In [ ]:
# Modelos antes do SMOTE
df_resultados_iniciais = pd.read_csv("resultados_modelos_iniciais.csv")

# Modelos após SMOTE e ajuste de threshold
df_resultados_ajustados = pd.read_csv("resultados_modelos_ajustados.csv")

# Feature importance dos modelos finais (opcional)
df_feature_importance = pd.read_csv("feature_importance_modelo_final.csv")


## 3️⃣ Tabela Comparativa: Antes vs Depois

In [ ]:
# Unindo os dois DataFrames
df_comparativo = pd.merge(
    df_resultados_iniciais,
    df_resultados_ajustados,
    on="Modelo",
    suffixes=("_Antes", "_Depois")
)

# Visualização
df_comparativo


## 4️⃣ Gráficos Comparativos (Side by Side)

In [ ]:
# Comparação de ROC AUC
fig, ax = plt.subplots(figsize=(10, 6))
x = np.arange(len(df_comparativo['Modelo']))
largura = 0.35

plt.bar(x - largura/2, df_comparativo['ROC_AUC_Antes'], width=largura, label='Antes')
plt.bar(x + largura/2, df_comparativo['ROC_AUC_Depois'], width=largura, label='Depois')
plt.xticks(x, df_comparativo['Modelo'])
plt.ylabel('ROC AUC')
plt.title('Comparação do ROC AUC dos Modelos')
plt.legend()
plt.show()


## 5️⃣ Feature Importance (Top 10 variáveis mais relevantes)

In [ ]:
# Top 10 features
top_features = df_feature_importance.sort_values(by='Importancia', ascending=False).head(10)

# Visualização
plt.figure(figsize=(8,6))
sns.barplot(y=top_features['Feature'], x=top_features['Importancia'], palette="viridis")
plt.title("Top 10 Features mais relevantes (modelo final)")
plt.xlabel("Importância")
plt.ylabel("Variável")
plt.show()


## 6️⃣ Conclusão Executiva e Recomendações

## 🧠 Conclusão

- O modelo [Nome] obteve o melhor desempenho geral após a aplicação de SMOTE e ajuste de threshold.
- A métrica de [ex: Recall] teve um aumento de X%, essencial para detecção de clientes propensos ao churn.
- As variáveis mais influentes foram: [lista resumida].

## 📌 Recomendações Estratégicas

1. Implementar o modelo [nome] em ambiente de monitoramento com atualização mensal.
2. Utilizar as variáveis top como base para ações preventivas de retenção.
3. Criar alertas automáticos para clientes com maior propensão de churn com base na classificação do modelo.
